In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [3]:
df = pd.read_csv("../data/solutions.csv")
df.head()

,QCode,SolutionID,timeago,UserID,Status,TimeTaken,MemTaken,Language,SolutionUrl
0,A1,S11986970,07:58 PM 31/10/16,/users/ismail_2311,accepted,1.43,2M,C,/viewplaintext/11986970
1,A1,S11990433,07:51 PM 01/11/16,/users/mohit_15,wrong answer,0.00,2M,C,/viewplaintext/11990433
2,A1,S11990440,07:54 PM 01/11/16,/users/mohit_15,accepted,0.00,2M,C,/viewplaintext/11990440
3,A1,S11995888,12:17 PM 03/11/16,/users/saloni1907,compilation error,-,-,C++ 4.3.2,/viewplaintext/11995888
4,A1,S11995889,12:18 PM 03/11/16,/users/saloni1907,accepted,0.00,2.7M,C++ 4.3.2,/viewplaintext/11995889


In [14]:
df = df[df["Status"] == "accepted"]
group = df.groupby("QCode")
group_keys = group.groups.keys()

In [44]:
with open("./converted.txt", "w") as f:
    for g in group_keys:
        question = group.get_group(g)
        question = question[['TimeTaken', 'SolutionUrl']]
        question = question.sort_values("TimeTaken")
        question = question.drop_duplicates(subset=["TimeTaken"])
        question['SolutionUrl'] = 'https://www.codechef.com' + question['SolutionUrl']
        html = requests.get(question.iloc[0]["SolutionUrl"]).text
        soup = BeautifulSoup(html, 'html.parser')
        completion = soup.get_text()
        for _, (_, link) in question.iloc[1:].iterrows():
            html = requests.get(question.iloc[0]["SolutionUrl"]).text
            soup = BeautifulSoup(html, 'html.parser')
            prompt = "# Make this code more efficient\n" + soup.get_text()
            to_write = json.dumps({"prompt": prompt.replace('"', "\""), "completion": completion.replace('"', "\"")})
            f.write(to_write+"\n")


In [47]:
with open('./converted.txt', "r") as i:
    with open("./converted_no_duplicates.txt", "w") as o:
        uniqlines = list(set(i.readlines()))
        o.writelines(uniqlines)

In [48]:
len(group_keys)

339